In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import joblib
import sasc.config
import numpy as np
from collections import defaultdict
import pandas as pd
from sasc.modules.fmri_module import convert_module_num_to_voxel_num, add_stability_score
from sasc.config import FMRI_DIR, STORIES_DIR

# Read all the info from stories into a single pickle file

In [2]:
# load stuff
# double check all of these, intro paragraph may be the same...
output_file = join(sasc.config.RESULTS_DIR,
                   'processed', "pilot5_story_data.pkl")
story_mapping = {
    'roi/uts02___roi_may31___seed=5_best1': 'GenStory23.npy',
    'roi/uts02___roi_may31___seed=2_best2': 'GenStory24.npy',
    'roi/uts02___roi_may31___seed=7_best3': 'GenStory25.npy',
    'roi/uts02___roi_may31___seed=6_best4': 'GenStory26.npy',

    'qa/uts02___qa_may31___seed=1': 'GenStory27_resps.npy',
    'qa/uts02___qa_may31___seed=2': 'GenStory28_resps.npy',
    'qa/uts02___qa_may31___seed=3': 'GenStory29_resps.npy',
}

# cluster_neighbors = joblib.load(join(FMRI_DIR, "voxel_neighbors_and_pcs", "cluster_neighbors_v1.pkl"))
perfs = joblib.load(join(sasc.config.FMRI_DIR, 'sasc', 'rj_models',
                    'opt_model', 'new_setup_performance.jbl'))

# add keys
stories_data_dict = defaultdict(list)
for story_idx, story_name in enumerate(story_mapping.keys()):
    # add scalar story descriptions
    stories_data_dict["story_name_original"].append(story_name)
    stories_data_dict["story_setting"].append(story_name.split("/")[0])
    stories_data_dict["story_name_new"].append(story_mapping[story_name])
    stories_data_dict["story_text"].append(
        open(join(STORIES_DIR, story_name, "story.txt"), "r").read()
    )
    prompts_paragraphs = joblib.load(
        join(STORIES_DIR, story_name, "prompts_paragraphs.pkl")
    )

    # add paragraph-level descriptions
    timings = pd.read_csv(
        join(STORIES_DIR, story_name, "timings_processed.csv"))
    # add offset to timings
    timings['time_running'] += 10  # -= 5  # += 6
    stories_data_dict["timing"].append(timings)
    stories_data_dict["prompts"].append(prompts_paragraphs["prompts"])
    stories_data_dict["paragraphs"].append(prompts_paragraphs["paragraphs"])

    # add paragraph-level metadata
    # rows
    # rows = pd.read_csv(join(STORIES_DIR, story_name, "rows.csv"))
    story_metadata_per_paragraph = pd.read_pickle(
        join(STORIES_DIR, story_name, "rows.pkl"))
    # story_metadata_per_paragraph["voxel_num"] = story_metadata_per_paragraph.apply(
    # lambda row: convert_module_num_to_voxel_num(
    # row["module_num"], row["subject"]),
    # axis=1,
    # )
    # story_metadata_per_paragraph["stability_score"] = story_metadata_per_paragraph.apply(
    # lambda row: add_stability_score(row["module_num"], row["subject"]),
    # axis=1,
    # )
    # story_metadata_per_paragraph = story_metadata_per_paragraph[
    #     [
    #         "expl",
    #         "module_num",
    #         "top_explanation_init_strs",
    #         "subject",
    #         "fmri_test_corr",
    #         "stability_score",
    #         # "top_score_synthetic",
    #         "top_score_normalized",
    #         "roi_anat",
    #         "roi_func",
    #         "voxel_num",
    #     ]
    # ]
    # story_metadata_per_paragraph['test_corr_new'] = story_metadata_per_paragraph['voxel_num'].apply(
    #     lambda x: perfs[x])
    # rows['cluster_nums'] = rows['voxel_num'].map(cluster_neighbors)
    stories_data_dict["rows"].append(story_metadata_per_paragraph)

joblib.dump(stories_data_dict, output_file)

['/home/chansingh/automated-explanations/results/processed/pilot5_story_data.pkl']

### extra exploration

In [5]:
metadata = pd.read_pickle('../0_voxel_select/rows_roi_uts02_may31.pkl')
metadata

,roi,expl,top_ngrams_module_correct,stability_score,subject,voxel_nums,prompt_suffix
0,RSC,Travel and location names,"[drove from vermont, to washington, in manhatt...",0.727578,UTS02,"[26313, 26368, 26369, 26370, 26423, 26424, 264...",
1,OPA,Direction and location descriptions,"[onto the railing, towards the river, onto the...",0.683813,UTS02,"[24026, 27029, 27030, 27031, 27075, 27076, 270...",Avoid mentioning any specific location names ...
2,PPA,Scenes and settings,"[mile of cornfields, the windowsill, the rolli...",0.417527,UTS02,"[9579, 9580, 9634, 11900, 11901, 11902, 11903,...",Avoid mentioning any specific location names ...
3,IPS,Descriptive elements of scenes or objects,"[there were slats, four connected squares, in ...",0.635485,UTS02,"[52728, 52729, 52730, 52781, 52782, 52783, 528...",Avoid mentioning any locations.
4,pSTS,Verbal interactions,"[said excuse me, says excuse me, room went sil...",0.601735,UTS02,"[26011, 26067, 26068, 26123, 26124, 26125, 291...",Avoid mentioning any locations.
5,sPMv,Time and numbers,"[one, forty, april nineteen forty, was sixteen...",0.564742,UTS02,"[71004, 71005, 71006, 71024, 71025, 71053, 710...",Avoid mentioning any locations.
6,EBA,Body parts,"[wraps his arms, lifted her dress, arms flaili...",0.623958,UTS02,"[10330, 10331, 10370, 10371, 10410, 12693, 126...",Avoid mentioning any locations.
7,OFA,Personal growth and reflection,"[of my childhood, newfound self esteem, so my ...",0.574794,UTS02,"[8133, 8161, 8189, 8190, 8221, 8222, 8223, 822...",Avoid mentioning any locations.
8,RSC_only,Location names,"[florida, israel, london, marrakesh, indianapo...",0.727578,UTS02,"[26313, 26368, 26369, 26370, 26423, 26424, 264...",
9,OPA_only,Spatial positioning and directions,"[towards the ceiling, onto the railing, feet h...",0.683813,UTS02,"[24026, 27029, 27030, 27031, 27075, 27076, 270...",Avoid mentioning any specific location names ...


In [6]:
metadata['expl'].unique().shape

(11,)

In [ ]:
story_metadata_per_paragraph['expl'].unique().shape

In [ ]:
# merge
assert story_metadata_per_paragraph['expl'].unique(
).shape == metadata['expl'].unique().shape
story_metadata_per_paragraph = pd.merge(
    story_metadata_per_paragraph, metadata[['expl', 'roi', 'voxel_nums', 'stability_score']], on=['expl'], how='inner', )

In [ ]:
story_metadata_per_paragraph